In [1]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import math
import os
from operator import itemgetter, attrgetter, methodcaller
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using Theano backend.


# Output of song

In [2]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()

# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

### TODOS
- try to predict hold/roll there (would need to train on later beat info as well)
- generate percent of single double notes etc with a nn

In [3]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.note_classes = pd.read_csv('generated_data/{0}_note_classes_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [4]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_note_classes_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [5]:
beats_to_track = 48
num_classes_one_note = 4
num_classes_two_note = 6
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}
class_reverse_map_one_note = ['1000', '0100', '0010', '0001']

class_map_two_note = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}
class_reverse_map_two_note = ['1001', '0110', '1100', '1010', '0101', '0011']

note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

song_padding = beats_to_track * 2
song_end_padding = beats_to_track * 2
important_indices = [1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
important_indices_classes = [-96, -84, -72, -60, -48, -36, -24, -12, 0, 1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
def get_features(index, features, note_classes):
    indices = [index + song_padding - i for i in important_indices]
    indices_classes = [index + song_padding - i for i in important_indices_classes]
    return np.concatenate((np.array([note_classes[i] for i in indices_classes]).flatten(), np.array([features[i] for i in indices]).flatten()), axis = 0)

In [6]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(100, input_dim=776, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

# Classes
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [7]:
def get_model_class_for_notes(row):
    note_counts = [row.count(note_type) for note_type in note_types]
    (blank, steps, mines, hold_starts, roll_starts, hold_ends) = note_counts
    
    model_classes = []
    if steps + hold_starts + roll_starts == 1:
        model_classes.append(1)

    if steps + hold_starts + roll_starts == 2:
        model_classes.append(2)
        
    if steps + hold_starts + roll_starts > 2:
        model_classes.append(3)
        
    if hold_starts > 0:
        model_classes.append(4)
        
    if roll_starts > 0:
        model_classes.append(5)
        
    if mines > 0:
        model_classes.append(6)
        
    return model_classes

targets = ['0', '1', '1', '1', '2', '4', 'M']
ammounts = [0, 1, 2, 3, 1, 1, 1]
def get_model_output_for_class(model_class, row):
    if model_class == 1 or model_class == 2 or model_class == 3:
        return [int(char == '1' or char == '2' or char == '4') for char in row]
    if model_class == 4:
        return [int(char == '2') for char in row]
    if model_class == 5:
        return [int(char == '4') for char in row]
    if model_class == 6:
        return [int(char == 'M') for char in row]

In [8]:
X = [[] for i in range(7)]
y = [[] for i in range(7)]
for key in list(songs.keys()):
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), songs[key].note_classes, ([[1, 0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
    notes = np.concatenate((([['0000']] * song_padding), songs[key].notes), axis = 0)
    if abs(len(note_classes) - len(notes) > 250):
        print ('Lengths dont match for {0}'.format(key))
        print ('{0} vs {1}'.format(len(note_classes), len(notes)))
        continue
    length = min(len(note_classes) - song_padding - song_end_padding, len(notes) - song_padding)
    features = np.array([get_features_for_row(notes[i][0]) for i in range(-song_padding, length)])
    for i in range(length):
        row = notes[i + song_padding][0]
        model_classes = get_model_class_for_notes(row)
        for model_class in model_classes:
            X[model_class].append(get_features(i, features, note_classes))
            y[model_class].append(get_model_output_for_class(model_class, row))

X = [np.array(X_for_class) for X_for_class in X]
y = [np.array(y_for_class) for y_for_class in y]

In [9]:
[len(X[i]) for i in range(7)]

[0, 82708, 6598, 36, 6484, 228, 3985]

In [10]:
models = []
models.append(None)
for i in range(1, 7):
    model = build_model(4)
    model.fit(X[i], y[i], nb_epoch=3, batch_size=math.ceil(len(X[i]) / 10000))
    model.save('models/write_notes_model_{0}.h5'.format(i))
    models.append(model)
# Dense 1000: 63s - loss: 1.3934 - acc: 0.2707
# Dense 550: 53s - loss: 1.3903 - acc: 0.2703
# Dense 100: 30s - loss: 1.3950 - acc: 0.2699

Epoch 1/3
82708/82708 [==============================] - 114s - loss: 1.3873 - acc: 0.2706   
Epoch 2/3
82708/82708 [==============================] - 108s - loss: 1.3810 - acc: 0.2760   
Epoch 3/3
82708/82708 [==============================] - 96s - loss: 1.3808 - acc: 0.2750    
Epoch 1/3
6598/6598 [==============================] - 61s - loss: 2.7705 - acc: 0.1954    
Epoch 2/3
6598/6598 [==============================] - 67s - loss: 2.7707 - acc: 0.1593    
Epoch 3/3
6598/6598 [==============================] - 70s - loss: 2.7709 - acc: 0.2149    
Epoch 1/3
36/36 [==============================] - 0s - loss: 4.5057 - acc: 0.0000e+00     
Epoch 2/3
36/36 [==============================] - 0s - loss: 4.5054 - acc: 0.0000e+00     
Epoch 3/3
36/36 [==============================] - 0s - loss: 4.5052 - acc: 0.0000e+00     
Epoch 1/3
6484/6484 [==============================] - 72s - loss: 1.4875 - acc: 0.2483    
Epoch 2/3
6484/6484 [==============================] - 73s - loss: 1.4874 

In [11]:
def get_average_for_class(note_classes, i, class_num):
    surrounding_beats = note_classes[max(i - 24,0):min(i + 24,len(note_classes))]
    return sum([beat[class_num] for beat in surrounding_beats]) / float(len(surrounding_beats))

def normalize_row(note_classes, i):
    return [note_classes[i][class_num] - get_average_for_class(note_classes, i, class_num) for class_num in range(7)]
    
def normalize_classes(note_classes):
    return [normalize_row(note_classes, i) for i in range(len(note_classes))]

In [12]:
pattern = ['1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010', '1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010']
cutoff_per_class = [0, 0.9, 0.97, 0.99, 0.99, 0.99, 0.99]
def get_output(song):
    predicted_notes = []
    normalized_note_classes = song.note_classes #normalize_classes(song.note_classes)
    sortedLists = [sorted(normalized_note_classes, key=itemgetter(i)) for i in range(7)]
    num_samples = len(song.note_classes)
    cutoffs = [sortedLists[i][int(num_samples * cutoff_per_class[i])][i] for i in range(7)]
    
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), song.note_classes, ([[1, 0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
    dummy_rows = [row for eigth in pattern for row in [eigth] + ['0000'] * 5]
    features = [get_features_for_row(row) for row in dummy_rows]
    for i in range(num_samples):
        note_class = song.note_classes[i]
        normalized_note_class = normalized_note_classes[i]
        prediction = '0000'
        for i in range(1, 7):
            if normalized_note_class[i] > cutoffs[i]:
                X_row = get_features(len(features) - song_padding, features, note_classes)
                prediction_values = models[i].predict(np.array([X_row]))[0]
                cutoff = sorted(prediction_values)[-ammounts[i]]
                prediction = ''.join([targets[i] if value >= cutoff else '0' for value in prediction_values])

        predicted_notes.append(prediction)
        features.append(get_features_for_row(prediction))
    return predicted_notes

In [13]:
def step_song_by_name(name):
    song = songs['In The Groove~{0}'.format(name)]
    song.predicted_notes = get_output(song)
    step_song(song)

step_song_by_name('Anubis')
step_song_by_name('Bend Your Mind')
step_song_by_name('Boogie Down')
step_song_by_name('Bouff')
step_song_by_name('Bubble Dancer')

In [99]:
predicted_notes = []
normalized_note_classes = song.note_classes
sortedLists = [sorted(normalized_note_classes, key=itemgetter(i)) for i in range(7)]
num_samples = len(song.note_classes)
cutoffs = [sortedLists[i][int(num_samples * cutoff_per_class[i])][i] for i in range(7)]

note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), song.note_classes, ([[1, 0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
dummy_rows = [row for eigth in pattern for row in [eigth] + ['0000'] * 5]
features = [get_features_for_row(row) for row in dummy_rows]
for i in range(num_samples):
    note_class = song.note_classes[i]
    normalized_note_class = normalized_note_classes[i]
    prediction = '0000'
    for i in range(1, 7):
        if normalized_note_class[i] > cutoffs[i]:
            X_row = get_features(len(features) - song_padding, features, note_classes)
            prediction_values = models[i].predict(np.array([X_row]))[0]
            cutoff = sorted(prediction_values)[-ammounts[i]]
            prediction = ''.join([targets[i] if value >= cutoff else '0' for value in prediction_values])

    predicted_notes.append(prediction)
    features.append(get_features_for_row(prediction))

In [100]:
cutoffs

[0.15125609934329987,
 0.57552510499954224,
 0.11418217420578004,
 9.7502605058252798e-05,
 0.091162502765655531,
 0.00068649463355541218,
 0.033921748399734497]